## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from kmodes.kmodes import KModes

## Limpeza e pré processamento

### Carregamento

In [ ]:
desired_cols = [
  "CS_GESTANT",
  "ANO_NASC",

  "CS_RACA",
  "CS_ESCOL_N",

  "FEBRE",    # Sintomas
  "MIALGIA",
  "CEFALEIA",
  "EXANTEMA",
  "VOMITO",
  "NAUSEA",
  "DOR_COSTAS",
  "CONJUNTVIT",
  "ARTRITE",
  "ARTRALGIA",
  "PETEQUIA_N",
  "LEUCOPENIA",
  "LACO",
  "DOR_RETRO",

  "DIABETES",   # Comorbidades
  "HEMATOLOG",
  "HEPATOPAT",
  "RENAL",
  "HIPERTENSA",
  "ACIDO_PEPT",
  "AUTO_IMUNE",

  "HOSPITALIZ", # Desfecho
  "CLASSI_FIN", 
  "EVOLUCAO",
]

# Carregando dataset
DATA_PATH = "./data/original-2/DENGBR24.csv"


chunks = []

for chunk in pd.read_csv(DATA_PATH, low_memory=False, usecols=desired_cols, chunksize=500_000):
  filtered = chunk[chunk["CS_GESTANT"].isin([1, 2, 3])]
  chunks.append(filtered)


df = pd.concat(chunks, ignore_index=True)
df.head()

In [ ]:
df.shape

### Pré processamento e transformação

#### Sintomas e Comorbidades

In [ ]:
sint_com_cols = [
  "FEBRE",    # Sintomas
  "MIALGIA",
  "CEFALEIA",
  "EXANTEMA",
  "VOMITO",
  "NAUSEA",
  "DOR_COSTAS",
  "CONJUNTVIT",
  "ARTRITE",
  "ARTRALGIA",
  "PETEQUIA_N",
  "LEUCOPENIA",
  "LACO",
  "DOR_RETRO",

  "DIABETES",   # Comorbidades
  "HEMATOLOG",
  "HEPATOPAT",
  "RENAL",
  "HIPERTENSA",
  "ACIDO_PEPT",
  "AUTO_IMUNE",
]

In [ ]:
df[sint_com_cols] = df[sint_com_cols].fillna(0)
df[sint_com_cols] = df[sint_com_cols].replace(2,0)
df[sint_com_cols] = df[sint_com_cols].astype(int)

#### Raça, escolaridade e gestação e idade

In [ ]:
# raça
df["CS_RACA"] = df["CS_RACA"].replace(9,np.nan)
df = df.dropna(subset=["CS_RACA"])

# escolaridade
# df["CS_ESCOL_N"] = df["CS_ESCOL_N"].fillna(9)
df["CS_ESCOL_N"] = df["CS_ESCOL_N"].replace(9,np.nan)
df= df.dropna(subset=["CS_ESCOL_N"])

# idade
df = df.dropna(subset=["ANO_NASC"])
current_year = 2024
df["IDADE"] = current_year - df["ANO_NASC"]
df["IDADE"] = df["IDADE"].astype(int)

# df["FAIXA_IDADE"] = pd.cut(df["IDADE"], bins=[0, 10, 20, 30, 40, 60, 100], labels=["0-10", "11-20", "21-30", "31-40", "41-60", "60+"])
df["FAIXA_IDADE"] = pd.cut(df["IDADE"], bins=[15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 70], labels=["15-20", "21-25", "26-30", "31-35", "36-40", "41-45", "46-50", "51-55", "56-60", "60+"])
df = df.dropna(subset=["FAIXA_IDADE"])

socio_targets = [
  "CS_RACA",
  "CS_ESCOL_N",
  "CS_GESTANT",
  "FAIXA_IDADE"
]

df_dummies = pd.get_dummies(df, columns=socio_targets, prefix=socio_targets)
df_dummies.head()


## Clusterização

In [ ]:
clustering_cols = [

  "FEBRE",    # Sintomas
  "MIALGIA",
  "CEFALEIA",
  "EXANTEMA",
  "VOMITO",
  "NAUSEA",
  "DOR_COSTAS",
  "CONJUNTVIT",
  "ARTRITE",
  "ARTRALGIA",
  "PETEQUIA_N",
  "LEUCOPENIA",
  "LACO",
  "DOR_RETRO",

  "DIABETES",   # Comorbidades
  "HEMATOLOG",
  "HEPATOPAT",
  "RENAL",
  "HIPERTENSA",
  "ACIDO_PEPT",
  "AUTO_IMUNE",

  "CS_ESCOL_N_1.0", # escolaridade
  "CS_ESCOL_N_2.0",
  "CS_ESCOL_N_3.0",
  "CS_ESCOL_N_4.0",
  "CS_ESCOL_N_5.0",
  "CS_ESCOL_N_6.0",
  "CS_ESCOL_N_7.0",
  "CS_ESCOL_N_8.0",
  # "CS_ESCOL_N_9.0",

  "CS_RACA_1.0",  # raça
  "CS_RACA_2.0",
  "CS_RACA_3.0",
  "CS_RACA_4.0",
  "CS_RACA_5.0",

  "CS_GESTANT_1.0", # periodo gestacional
  "CS_GESTANT_2.0",
  "CS_GESTANT_3.0",

  "FAIXA_IDADE_15-20",
  "FAIXA_IDADE_21-25",
  "FAIXA_IDADE_26-30",
  "FAIXA_IDADE_31-35",
  "FAIXA_IDADE_36-40",
  "FAIXA_IDADE_41-45",
  "FAIXA_IDADE_46-50",
  "FAIXA_IDADE_51-55",
  "FAIXA_IDADE_56-60",
  "FAIXA_IDADE_60+"
]

df_clustering = df_dummies[clustering_cols]
df_clustering.head()

In [ ]:
costs = []
K_range = range(1, 11)

for k in K_range:
    km = KModes(n_clusters=k, init='Huang', n_init=1, verbose=1)
    km.fit_predict(df_clustering)
    costs.append(km.cost_)

plt.plot(K_range, costs, marker='o')
plt.xlabel('Número de clusters (K)')
plt.ylabel('Custo (Distância intra-cluster)')
plt.title('Método do Cotovelo para K-Modes')
plt.grid()
plt.show()

In [ ]:
# Crie o modelo com K=5 e uma inicialização robusta
km = KModes(n_clusters=4, init='Huang', n_init=3, verbose=1)

# Ajuste o modelo aos dados completos e obtenha os clusters
clusters = km.fit_predict(df_clustering)  # df = seu dataframe categórico

# Agora você pode adicionar os clusters no dataframe
df_clustering['cluster'] = clusters

In [ ]:
from sklearn.decomposition import TruncatedSVD

X = df_clustering.values  # pode ser matriz esparsa, ex: scipy.sparse.csr_matrix

svd = TruncatedSVD(n_components=2, random_state=1)
X_svd = svd.fit_transform(X)

plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_svd[:, 0], X_svd[:, 1], c=df_clustering['cluster'], cmap='tab10', s=10)
plt.legend(*scatter.legend_elements(), title="Clusters")
plt.title("Visualização dos clusters com TruncatedSVD")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.show()

In [ ]:
# TruncatedSVD para 3 componentes
svd = TruncatedSVD(n_components=3, random_state=1)
X_svd_3d = svd.fit_transform(X)

# Plot 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(
    X_svd_3d[:, 0], X_svd_3d[:, 1], X_svd_3d[:, 2],
    c=df_clustering['cluster'], cmap='tab10', s=15, alpha=0.8
)

ax.set_title("Visualização 3D dos clusters com TruncatedSVD")
ax.set_xlabel("Componente 1")
ax.set_ylabel("Componente 2")
ax.set_zlabel("Componente 3")

legend1 = ax.legend(*scatter.legend_elements(), title="Clusters")
ax.add_artist(legend1)

plt.show()

In [ ]:
df["cluster"] = clusters
df["cluster"].value_counts()

In [ ]:
df.groupby('cluster')['IDADE'].mean().plot(kind='bar')
plt.title('Média de Idade por Cluster')
plt.ylabel('Idade média')
plt.show()

In [ ]:
import seaborn as sns

cluster_profile = df_clustering.groupby('cluster').mean()

plt.figure(figsize=(10, 20))
sns.heatmap(cluster_profile.T, cmap='rocket_r', annot=False, fmt=".2f")
plt.title('Proporção média das categorias por cluster')
plt.xlabel('Categorias One-Hot')
plt.ylabel('Cluster')
plt.show()

In [ ]:
hospitaliz_table = pd.crosstab(df["cluster"], df["HOSPITALIZ"])
hospitaliz_table.columns = ["Sim", "Não", "Ignorado"]
hospitaliz_prop = hospitaliz_table.div(hospitaliz_table.sum(axis=1), axis=0)
hospitaliz_prop = hospitaliz_prop * 100

plt.figure(figsize=(8, 6))
sns.heatmap(hospitaliz_prop, annot=hospitaliz_prop.round(2).astype(str) + '%', cmap="OrRd", fmt="")
plt.title("Proporção de Tipos de Hospitalização por Cluster")
plt.xlabel("Tipo de Hospitalização")
plt.ylabel("Cluster")
plt.show()

In [ ]:
evolution_table = pd.crosstab(df["cluster"], df["EVOLUCAO"])
evolution_table.columns = ["cura", "óbito pelo agravo", "óbito por outras causas", "ignorado"]
evolution_prop = evolution_table.div(evolution_table.sum(axis=1), axis=0)
evolution_prop = evolution_prop * 100

plt.figure(figsize=(8, 5))
sns.heatmap(evolution_prop, annot=evolution_prop.round(2).astype(str) + '%', cmap="OrRd", fmt="")
plt.title("Proporção de Tipos de Evolução por Cluster")
plt.xlabel("Tipo de Evolução")
plt.ylabel("Cluster")
plt.show()

In [ ]:
classi_table = pd.crosstab(df["cluster"], df["CLASSI_FIN"])
classi_table.columns = ["inconclusivo", "dengue", "dengue alarmante", "dengue grave"]
classi_prop = classi_table.div(classi_table.sum(axis=1), axis=0)
classi_prop = classi_prop * 100

plt.figure(figsize=(8, 5))
sns.heatmap(classi_prop, annot=classi_prop.round(2).astype(str) + '%', cmap="rocket_r", fmt="")
plt.title("Proporção de Tipos de Dengue por Cluster")
plt.xlabel("Tipo de Evolução")
plt.ylabel("Cluster")
plt.show()

In [ ]:
raca_table = pd.crosstab(df["cluster"], df["CS_RACA"])
raca_prop = raca_table.div(raca_table.sum(axis=1), axis=0)
raca_prop.columns = ["branca", "preta", "amarela", "parda", "indigena"]
raca_prop = raca_prop * 100

plt.figure(figsize=(8, 5))
sns.heatmap(raca_prop, annot=raca_prop.round(2).astype(str) + '%', cmap="OrRd", fmt="")
plt.title("Proporção de Raças por Cluster")
plt.xlabel("Raças")
plt.ylabel("Cluster")
plt.show()

In [ ]:
escol_table = pd.crosstab(df["cluster"], df["CS_ESCOL_N"])
escol_prop = escol_table.div(escol_table.sum(axis=1), axis=0)
# escol_prop.columns = ["branca", "preta", "amarela", "parda", "indigena"]
escol_prop = escol_prop * 100

plt.figure(figsize=(15, 5))
sns.heatmap(escol_prop, annot=escol_prop.round(2).astype(str) + '%', cmap="OrRd", fmt="")
plt.title("Proporção de Escolaridade por Cluster")
plt.xlabel("Nível de Escolaridade")
plt.ylabel("Cluster")
plt.show()

In [ ]:
gestant_table = pd.crosstab(df["cluster"], df["CS_GESTANT"])
gestant_prop = gestant_table.div(gestant_table.sum(axis=1), axis=0)
gestant_prop.columns = ["primeiro", "segundo", "terceiro"]
gestant_prop = gestant_prop * 100

plt.figure(figsize=(15, 5))
sns.heatmap(gestant_prop, annot=gestant_prop.round(2).astype(str) + '%', cmap="OrRd", fmt="")
plt.title("Proporção de Período Gestacional por Cluster")
plt.xlabel("Trimestres")
plt.ylabel("Cluster")
plt.show()